In [ ]:
import glob
import os
import suspect
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [12, 7]

# Extract data from LCModel COORD files

In [ ]:
coords = glob.glob('../derivatives/LCModel_MM/*/*/*/*.coord')
coords

In [ ]:
len(coords)

In [ ]:
d = dict()
for i in coords:
    filename=os.path.basename(i).strip('.coord')
    loc = filename.split('_')[2].split('-')[-1]
    sub = filename.split('_')[0].split('-')[-1]
    d['{}_{}'.format(sub,loc)] = suspect.io.lcmodel.read_coord(i)['metabolite_fits']

In [ ]:
suspect.io.lcmodel.read_coord(coords[2])['metabolite_fits']

In [ ]:
dd = dict()
for i in d:
    dd[i] = dict()
    for j in d[i]:
        dd[i][j] = d[i][j]['concentration']

In [ ]:
conc_df = pd.DataFrame.from_dict(dd).transpose()

In [ ]:
conc_df = conc_df[[i for i in conc_df.columns if i.startswith('M')]]

In [ ]:
conc_df

In [ ]:
conc_df['sub_id'] = conc_df.index

In [ ]:
conc_df['ROI'] = conc_df.sub_id.apply(lambda x: x.split('_')[-1]) 
conc_df['sub_id'] = conc_df.sub_id.apply(lambda x: x.split('_')[0]) 

In [ ]:
corr.to_csv('/home/orco/data/MacroMols/data/MM_concs_3T.csv')
conc_df.head()

In [ ]:
ax = sns.violinplot(data=conc_df)

In [ ]:
corr = conc_df.corr(method='spearman')
corr

In [ ]:
corr.to_csv('/home/orco/data/MacroMols/data/corrs_3T.csv')

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=np.bool))

In [ ]:
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, center=0.56, vmin=0,
            square=True, annot=True, fmt='1.2f', linewidths=.5, cbar_kws={"shrink": 0.5})

In [ ]:
#Ref: https://stackoverflow.com/questions/52741236/how-to-calculate-p-values-for-pairwise-correlation-of-columns-in-pandas
from scipy.stats import pearsonr, spearmanr

def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

pvalues_df = conc_df.corr(method=spearmanr_pval)

In [ ]:
from sklearn.linear_model import LinearRegression

def reg_coef(x,y):
    X = [np.array([i]) for i in x]
    model = LinearRegression()
    model.fit(X,y)
    return model.coef_

slopes_df = conc_df.corr(method=reg_coef)
diagonal = mask.copy()
mask = (pvalues_df >= 0.01) | (diagonal)

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(slopes_df, mask=mask, cmap=cmap, center=0.70, vmin=0,
            square=True, annot=True, fmt='1.2f', linewidths=.5, cbar_kws={"shrink": 0.5})